<a href="https://colab.research.google.com/github/pravallika89ch/alignn/blob/main/internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!pip install -q condacolab
import condacolab
condacolab.install()


✨🍰✨ Everything looks OK!


In [17]:
import os
os.environ["MPLBACKEND"] = "Agg"


In [18]:
!conda install -y numpy=1.23.5 pandas=1.5.3 python=3.10
!pip install jarvis-tools alignn torchdata dgl==1.1.1



Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / failed

SpecsConfigurationConflictError: Requested specs conflict with configured specs.
  requested specs: 
    - numpy=1.23.5
    - pandas=1.5.3
    - python=3.10
  pinned specs: 
    - cuda-version=12
    - python=3.11
    - python_abi=3.11[build=*cp311*]
Use 'conda config --show-sources' to look for 'pinned_specs' and 'track_features'
configuration parameters.  Pinned specs may also be defined in the file
/usr/local/conda-meta/pinned.




In [19]:
import pandas as pd
import json
import os
from jarvis.core.atoms import Atoms

# Set your folder paths
csv_path = "/content/drive/MyDrive/Internship/tmdc-co2-meoh/dataset/opentmdc/co2.vaspfiles/id_prop.csv"
vasp_folder = "/content/drive/MyDrive/Internship/tmdc-co2-meoh/dataset/opentmdc/co2.vaspfiles"
output_json = "co2_dataset.json"

# Load CSV
df = pd.read_csv(csv_path)

# Prepare dataset list
dataset = []
for i, row in df.iterrows():
    vasp_file = os.path.join(vasp_folder, row['id'])
    atoms = Atoms.from_poscar(vasp_file)
    dataset.append({
        "structure": atoms.to_dict(),
        "target": float(row['adsorption_energy (eV)']),
        "jid": row['id']
    })

# Save JSON
with open(output_json, "w") as f:
    json.dump(dataset, f, indent=2)

print("Dataset JSON created:", output_json)

Dataset JSON created: co2_dataset.json


In [20]:
import json

# Load the original file
with open("co2_dataset.json", "r") as f:
    data = json.load(f)

# Fix structure ➝ atoms
for entry in data:
    entry["atoms"] = entry.pop("structure")

# Save it back
with open("co2_dataset_fixed.json", "w") as f:
    json.dump(data, f, indent=2)

print("Fixed file written as co2_dataset_fixed.json")

Fixed file written as co2_dataset_fixed.json


In [21]:
import json, random

with open("co2_dataset_fixed.json") as f:
    data = json.load(f)

# Shuffle for randomness
random.shuffle(data)

# Split
train, val, test = data[:30], data[30:33], data[33:36]

# Save
with open("train.json", "w") as f: json.dump(train, f, indent=2)
with open("val.json", "w") as f: json.dump(val, f, indent=2)
with open("test.json", "w") as f: json.dump(test, f, indent=2)

print("✅ train/val/test JSON files created.")

✅ train/val/test JSON files created.


In [22]:
import json

with open("train.json", "r") as f:
    train_data = json.load(f)

print(f"✅ Total training samples: {len(train_data)}")


✅ Total training samples: 30


In [24]:
import os
import json
import lmdb
import pickle
from tqdm import tqdm

# Paths
json_file = "co2_dataset_fixed.json"
lmdb_path = "co2_lmdb_dataset"

# Load JSON
with open(json_file, "r") as f:
    data = json.load(f)

# Prepare LMDB directory
os.makedirs(lmdb_path, exist_ok=True)

env = lmdb.open(lmdb_path, map_size=1 << 40)

with env.begin(write=True) as txn:
    for idx, item in tqdm(enumerate(data), total=len(data)):
        txn.put(str(idx).encode(), pickle.dumps(item))

    # Save length
    txn.put(b"__len__", pickle.dumps(len(data)))
    # Save keys
    keys = [str(i).encode() for i in range(len(data))]
    txn.put(b"__keys__", pickle.dumps(keys))

env.close()

print(f"✅ LMDB saved at: {lmdb_path}")


100%|██████████| 36/36 [00:00<00:00, 12915.49it/s]

✅ LMDB saved at: co2_lmdb_dataset


In [36]:
config= {
  "version": "1.0",
  "dataset": "co2_lmdb_dataset",
  "dataset": "user_data",
  "use_lmdb": True,
  "target": "target",
  "id_tag": "jid",
  "atom_features": "cgcnn",
  "neighbor_strategy": "k-nearest",
  "dtype": "float32",
  "random_seed": 42,
  "n_train": None,
  "n_val": None,
  "n_test": None,
  "train_ratio": 0.8,
  "val_ratio": 0.1,
  "test_ratio": 0.1,
  "epochs": 100,
  "batch_size": 16,
  "weight_decay": 1e-5,
  "learning_rate": 0.001,
  "warmup_steps": 2000,
  "criterion": "mse",
  "optimizer": "adamw",
  "scheduler": "onecycle",
  "pin_memory": False,
  "save_dataloader": False,
  "write_checkpoint": True,
  "write_predictions": True,
  "store_outputs": True,
  "progress": True,
  "log_tensorboard": False,
  "standard_scalar_and_pca": False,
  "use_canonize": True,
  "compute_line_graph": True,
  "num_workers": 2,
  "cutoff": 8.0,
  "cutoff_extra": 3.0,
  "max_neighbors": 12,
  "keep_data_order": True,
  "normalize_graph_level_loss": False,
  "distributed": False,
  "data_parallel": False,
  "n_early_stopping": None,
  "output_dir": "TMDC_Results",
  "model": {
    "name": "alignn_atomwise",
    "alignn_layers": 2,
    "gcn_layers": 2,
    "atom_input_features": 92,
    "edge_input_features": 80,
    "triplet_input_features": 40,
    "embedding_features": 64,
    "hidden_features": 64,
    "output_features": 1,
    "grad_multiplier": -1,
    "calculate_gradient": True,
    "atomwise_output_features": 0,
    "graphwise_weight": 1.0,
    "gradwise_weight": 1.0,
    "stresswise_weight": 0.0,
    "atomwise_weight": 0.0,
    "link": "identity",
    "zero_inflated": False,
    "classification": False,
    "force_mult_natoms": False,
    "energy_mult_natoms": True,
    "include_pos_deriv": False,
    "use_cutoff_function": False,
    "inner_cutoff": 3.0,
    "stress_multiplier": 1.0,
    "add_reverse_forces": True,
    "lg_on_fly": True,
    "batch_stress": True,
    "multiply_cutoff": False,
    "use_penalty": True,
    "extra_features": 0,
    "exponent": 5,
    "penalty_factor": 0.1,
    "penalty_threshold": 1.0,
    "additional_output_features": 0,
    "additional_output_weight": 0.0
  }
}


In [35]:
import json

with open("configfile.json", "w") as f:
    json.dump(config, f, indent=2)

print("✅ Config saved successfully!")


✅ Config saved successfully!


In [ ]:
%env MPLBACKEND=Agg


env: MPLBACKEND=Agg


In [ ]:
!git clone https://github.com/usnistgov/alignn.git

Cloning into 'alignn'...
remote: Enumerating objects: 5317, done.
remote: Counting objects: 100% (1638/1638), done.
remote: Compressing objects: 100% (550/550), done.
remote: Total 5317 (delta 1337), reused 1118 (delta 1087), pack-reused 3679 (from 3)
Receiving objects: 100% (5317/5317), 155.11 MiB | 29.58 MiB/s, done.
Resolving deltas: 100% (3243/3243), done.


In [ ]:
import os
os.environ["MPLBACKEND"] = "Agg"
import matplotlib
matplotlib.use("Agg")


In [44]:
!python /content/alignn/alignn/train.py /content/configfile.json



✅ alignn loaded from: /content/alignn/alignn/__init__.py
fatal: not a git repository (or any of the parent directories): .git
/content/alignn/alignn/train.py:79: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print("config:", config.dict())
config: {'version': 'NA', 'dataset': 'user_data', 'target': 'exfoliation_energy', 'atom_features': 'cgcnn', 'neighbor_strategy': 'k-nearest', 'id_tag': 'jid', 'dtype': 'float32', 'random_seed': 123, 'classification_threshold': None, 'n_val': 32, 'n_test': None, 'n_train': 32, 'train_ratio': 0.8, 'val_ratio': 0.1, 'test_ratio': 0.1, 'target_multiplication_factor': None, 'epochs': 10, 'batch_size': 16, 'weight_decay': 0.0, 'learning_rate': 0.01, 'filename': 'sample', 'warmup_steps': 2000, 'criterion': 'mse', 'optimizer': 'adamw', 'scheduler': 'onecycle', 'pin_memory': False, 'save_da